<a href="https://colab.research.google.com/github/Viveksapkale22/DeepLearning_Model_Evaluators/blob/main/Gender_detection_model_testers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test-1 : haarcascade_frontalface_default + own gender detection model V13-mobilenetV3  

result :
- capture the face perfectly accuracy result 75-90

issue :
- can't track the face perfectly
- [chcek multiple face detection]
- [chcek try from distance]
- [chcek any miss-match issue chcek]

In [ ]:
!pip install tensorflow opencv-python-headless pillow

In [ ]:
import cv2
import numpy as np
from base64 import b64decode, b64encode
import PIL.Image
import io
from IPython.display import display, Javascript
from google.colab.output import eval_js
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
# Convert JavaScript webcam image to OpenCV BGR image
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# Convert overlay bounding boxes to base64 bytes for Colab
def bbox_to_bytes(bbox_array):
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='PNG')
    bbox_bytes = 'data:image/png;base64,{}'.format(b64encode(iobuf.getvalue()).decode('utf-8'))
    return bbox_bytes

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def video_stream_colab():
    js = Javascript('''
    var video; var div = null; var stream; var captureCanvas; var imgElement; var labelElement;
    var pendingResolve = null; var shutdown = false;
    function removeDom() {stream.getVideoTracks()[0].stop(); video.remove(); div.remove(); video = null; div = null; stream = null; imgElement = null; captureCanvas = null; labelElement = null;}
    function onAnimationFrame() { if(!shutdown){window.requestAnimationFrame(onAnimationFrame);} if(pendingResolve){var result=""; if(!shutdown){captureCanvas.getContext('2d').drawImage(video,0,0,640,480); result = captureCanvas.toDataURL('image/jpeg',0.8);} var lp=pendingResolve; pendingResolve=null; lp(result);} }
    async function createDom(){ if(div!==null){return stream;} div=document.createElement('div'); div.style.border='2px solid black'; div.style.padding='3px'; div.style.width='100%'; div.style.maxWidth='600px'; document.body.appendChild(div); const modelOut=document.createElement('div'); modelOut.innerHTML="<span>Status:</span>"; labelElement=document.createElement('span'); labelElement.innerText='No data'; labelElement.style.fontWeight='bold'; modelOut.appendChild(labelElement); div.appendChild(modelOut); video=document.createElement('video'); video.style.display='block'; video.width=div.clientWidth-6; video.setAttribute('playsinline',''); video.onclick=()=>{shutdown=true;}; stream=await navigator.mediaDevices.getUserMedia({video:{facingMode:"user"}}); div.appendChild(video); imgElement=document.createElement('img'); imgElement.style.position='absolute'; imgElement.style.zIndex=1; imgElement.onclick=()=>{shutdown=true;}; div.appendChild(imgElement); const instruction=document.createElement('div'); instruction.innerHTML='<span style="color:red;font-weight:bold;">Click video to stop demo</span>'; div.appendChild(instruction); instruction.onclick=()=>{shutdown=true;}; video.srcObject=stream; await video.play(); captureCanvas=document.createElement('canvas'); captureCanvas.width=640; captureCanvas.height=480; window.requestAnimationFrame(onAnimationFrame); return stream;}
    async function stream_frame(label,imgData){ if(shutdown){removeDom(); shutdown=false; return ''; } var preCreate=Date.now(); stream=await createDom(); var preShow=Date.now(); if(label!=""){labelElement.innerHTML=label;} if(imgData!=""){var videoRect=video.getClientRects()[0]; imgElement.style.top=videoRect.top+"px"; imgElement.style.left=videoRect.left+"px"; imgElement.style.width=videoRect.width+"px"; imgElement.style.height=videoRect.height+"px"; imgElement.src=imgData;} var preCapture=Date.now(); var result=await new Promise(function(resolve,reject){pendingResolve=resolve;}); shutdown=false; return {'create':preShow-preCreate,'show':preCapture-preShow,'capture':Date.now()-preCapture,'img':result};}
    ''')
    display(js)

def video_frame_colab(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

In [ ]:
# Preprocess face image for MobileNetV3 Small
def preprocess_image(img, target_size=(224,224)):
    img_resized = cv2.resize(img, target_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb / 255.0
    return np.expand_dims(img_normalized, axis=0)

# Predict gender using the model
def predict_gender(face_img, model):
    input_img = preprocess_image(face_img)
    pred = model.predict(input_img)[0][0]  # Assuming sigmoid output
    gender = 'Male' if pred > 0.5 else 'Female'
    confidence = pred if pred > 0.5 else 1 - pred
    return gender, confidence

In [ ]:
def colab_webcam_gender_detection(gender_model):
    """
    Run webcam in Google Colab, detect faces, and predict gender in real-time.
    Click on the video to stop the stream.
    """
    video_stream_colab()
    label_html = 'Detecting Faces...'
    bbox = ''

    while True:
        js_reply = video_frame_colab(label_html, bbox)
        if not js_reply:
            break

        img = js_to_image(js_reply["img"])
        bbox_array = np.zeros([480,640,4], dtype=np.uint8)

        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray)

        for (x,y,w,h) in faces:
            face_img = img[y:y+h, x:x+w]
            gender, conf = predict_gender(face_img, gender_model)
            label = f"{gender} {conf:.2f}"
            bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
            cv2.putText(bbox_array, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)

        bbox_array[:,:,3] = (bbox_array.max(axis=2)>0).astype(int)*255
        bbox = bbox_to_bytes(bbox_array)

In [ ]:

# Load your saved MobileNetV3 Small gender model
model_path = '/content/drive/MyDrive/Sem 7/Major project/best_gender_model_e12_v13.keras'
gender_model = load_model(model_path)

# Run webcam with gender detection
colab_webcam_gender_detection(gender_model)

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


/tmp/ipython-input-1371718878.py:10: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
